<a href="https://colab.research.google.com/github/djhsgfjk/collab/blob/main/Lab_8_%D0%A1%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

Разделить данные на обучающую, валидационную и тестовую выборки.

Нормализовать данные перед обучением. 

In [ ]:
img_transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))]
)

In [ ]:
train_data = datasets.MNIST(root='/data/notebook_files/',
                          download=True,
                          train=True,
                          transform=img_transform)

test_data = datasets.MNIST(root='/data/notebook_files/',
                        download=True,
                        train=False,
                        transform=img_transform)

val_data = datasets.MNIST(root='/data/notebook_files/',
                        download=True,
                        train=False,
                        transform=img_transform)

train_data_loader = torch.utils.data.DataLoader(train_data,
                                                batch_size=64,
                                               )
test_data_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=64,
                                              )
val_data_loader = torch.utils.data.DataLoader(val_data,
                                              batch_size=64,
                                              )


В качестве функций активации взять ReLU. 

In [ ]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 8, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(8, 10, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(10, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classification = nn.Sequential(
            nn.Linear(7 * 7 * 16, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 7 * 7 * 16)
        x = self.classification(x)
        return x

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=10, device="cpu"):
    for epoch in range(1, epochs + 1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item()
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output, targets)
            valid_loss += loss.data.item()
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],
                               targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'
              .format(epoch, training_loss, valid_loss, num_correct / num_examples))

In [ ]:
vgg = VGG()
vgg = vgg.to('cpu')
optimizer = optim.SGD(vgg.parameters(), lr=0.001, momentum=0.9)

В качестве целевой функцци взять nn.CrossEntropyLoss().

In [ ]:
train(vgg, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader)

Epoch: 1, Training Loss: 0.00, Validation Loss: 0.00, accuracy = 0.89
Epoch: 2, Training Loss: 0.00, Validation Loss: 0.00, accuracy = 0.89
Epoch: 3, Training Loss: 0.00, Validation Loss: 0.00, accuracy = 0.89
Epoch: 4, Training Loss: 0.00, Validation Loss: 0.00, accuracy = 0.98
Epoch: 5, Training Loss: 0.00, Validation Loss: 0.00, accuracy = 0.98
Epoch: 6, Training Loss: 0.00, Validation Loss: 0.00, accuracy = 0.99
Epoch: 7, Training Loss: 0.00, Validation Loss: 0.00, accuracy = 0.98
Epoch: 8, Training Loss: 0.00, Validation Loss: 0.00, accuracy = 0.99
Epoch: 9, Training Loss: 0.00, Validation Loss: 0.00, accuracy = 0.98
Epoch: 10, Training Loss: 0.00, Validation Loss: 0.00, accuracy = 0.98
